In [ ]:
import pandas as pd
import pydeck as pdk
import folium
import sys
from pathlib import Path
import json
import plotly.express as px

import os

sys.path.append(str(Path(os.getcwd()).parent))

from lib.constants import DATA_DIR

In [ ]:
with open(DATA_DIR/'osuked/wind_farm_data.json') as f:
    wind_farm_dict = json.load(f)

In [ ]:
wind_farm_dict.keys()

In [ ]:
# wind_farm_dict["WindFarms"] += wind_farm_dict["Winds"]

In [ ]:
wind_farm_dict["WindFarms"]

In [ ]:
all_farms = {}
for farm in wind_farm_dict["WindFarms"]:
    id_ = farm.get('Id', hash(( farm['Latitude'], farm['Longitude'])))
    all_farms[id_] = {"lat": farm['Latitude'], "lng": farm['Longitude']}

In [ ]:
len(all_farms)

In [ ]:
m = folium.Map(location=[54.763880, -0])
for farm_id, coords in all_farms.items():
    folium.Marker(
    [coords['lat'], coords['lng']], popup=f"<b>{farm_id}</b>"
    ).add_to(m)


In [ ]:
m

I think we may be missing a lot of onshore wind?

# Square this with curtailment

In [ ]:
df_curtailment = pd.read_feather(DATA_DIR/"outputs/curtailment-2022-01-01-2022-02-01.fthr")

In [ ]:
downsampled = []
for unit, data in df_curtailment.groupby("unit"):
    data = data.set_index("Time")
    data = data.resample("H").sum().reset_index()
    data['unit'] = unit
    downsampled.append(data)

In [ ]:
px.line(pd.concat(downsampled), x='Time', y='delta', color='unit')

In [ ]:
wind_farm_ids = [farm['Id'] for farm in wind_farm_dict["WindFarms"]]
sorted(wind_farm_ids)

These don't seem to be the same thing as the BM unit ids :) 

In [ ]:
df_curtailment['unit'].unique()

In [ ]:
data_dict = pd.read_csv(DATA_DIR/"osuked/dictionary_attributes.csv")

In [ ]:
data_dict[data_dict["id_type"]=="ngc_bmu_id"]


In [ ]:
data_dict[data_dict["dictionary_id"]==10000]


Our unit ID's dont seem to be in this data dictionary thing either, I think because we're using `sett` not `ngc`

In [ ]:
for unit_id in df_curtailment['unit'].unique():
    print(data_dict[data_dict["id"]==unit_id])

In [ ]:
def convert_sett_to_ngc_id(id_: str) -> str:
    return id_.split("_")[1]

Yep this works.

In [ ]:
for unit_id in df_curtailment['unit'].unique():
    is_present = len(data_dict[data_dict["id"]==convert_sett_to_ngc_id(unit_id)]) > 0
    print(unit_id, is_present)

And yet even after applying this conversion, only 3 of our IDs are present on the map

In [ ]:
set(wind_farm_ids).intersection(set([convert_sett_to_ngc_id(id_) for id_ in df_curtailment['unit'].unique()]))

So it looks like we need to get coords from this data dict monstrosity

In [ ]:
def get_coords_from_data_dict(sett_id: str) -> dict:
    ngc_id = convert_sett_to_ngc_id(sett_id)
    entries = data_dict[data_dict['id'] == ngc_id]
    print(entries['attribute'].unique())
    if len(entries) > 0:
        return dict(lat=entries.loc[entries['attribute']=='Latitude', "value"],
               lng = entries.loc[entries['attribute']=='Longitude', "value"])
    return None
    

In [ ]:
output = {}
for unit in df_curtailment['unit'].unique():
    output[unit]=get_coords_from_data_dict(unit)

In [ ]:
output

Unfortunately the lat/lng don't seem to be there either!

# Hacks
If we have all the wind units, we expect there to be a bimodal distribution with the stuff north of B6 being curtailed and everything below not...

In [ ]:
by_unit = df_curtailment.groupby('unit').sum().reset_index()
px.histogram(by_unit, x='delta', nbins=100)


In [ ]:
px.bar(by_unit, x='delta', y='unit')